In [34]:
import socket
import sys
import time
import numpy as np
from parseStatus import parseStatus

VISIONRADIUS = 200
KNOWN_WORMHOLE_LOC = []
def run(user, password, * commands):
    HOST, PORT = "codebb.cloudapp.net", 17429
    data = user + " " + password + "\n" + "\n".join(commands) + "\nCLOSE_CONNECTION\n"
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:

        sock.connect((HOST, PORT))
        sock.sendall(bytes(data, "utf-8"))
        sfile = sock.makefile()
        rline = sfile.readline()
        while rline:
            print(rline.strip())
            rline = sfile.readline()

def subscribe(user, password):
    HOST, PORT = "codebb.cloudapp.net", 17429
    data = user + " " + password + "\nSUBSCRIBE\n"
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(bytes(data, "utf-8"))
        sfile = sock.makefile()
        rline = sfile.readline()
        while rline:
            print(rline.strip())
            rline = sfile.readline()

In [35]:
def moveToPoint(xCur,yCur,xDest,yDest,mineFinding=False):
    
    speed = 1
    #STOP SHIP FIRST
    Moving = True
    run('ElectricBoogalo','kirtyhurty','BRAKE')
    while(Moving):
        stats = parseStatus()
        if(float(stats['dx']) < 0.5 and float(stats['dy']) <0.5):
            Moving = False

    #SET FIRST MOVE ANGLE
    angle = np.arctan((yDest-float(stats['y']))/(xDest-float(stats['x'])))
    if(xDest < float(stats['x'])):
        angle += np.pi
    print(angle)
    #CHECK FOR POSSIBLE WORMHOLES IN PATH
    check, angleChange = checkWormHoleCollision(xCur,yCur,xDest,yDest)
    if(check > -1):
        avoidWormHole(check, angleChange, angle, xDest,yDest)
    
    #TIME TO ACCELERATE
    Moving = True
    while(Moving):
        stats = parseStatus()
        angle = np.arctan((yDest-float(stats['y']))/(xDest-float(stats['x'])))
        if(xDest < float(stats['x'])):
            angle += np.pi
        print(angle)
        #CHECK FOR UNKNOWN WORMHOLES. IF DETECTED, BRAKE IMMEDIATELY.
        if(len(stats['wormholes']) > 0):
#             run('ElectricBoogalo','kirtyhurty','ACCELERATE ' + str(angle+np.pi) + ' 1')
#             time.sleep(2)
#             run('ElectricBoogalo','kirtyhurty','BRAKE')
            KNOWN_WORMHOLE_LOC.append(stats['wormholes'][0])
            print(KNOWN_WORMHOLE_LOC)
            #We know there's a wormhole in the way, probably, let's just do our best to avoid it
            check, angleChange = checkWormHoleCollision(float(stats['x']),float(stats['y']),xDest,yDest)
            avoidWormHole(check, angleChange, angle, xDest,yDest)

        run('ElectricBoogalo','kirtyhurty','ACCELERATE ' + str(angle) + ' ' + str(speed))
        if(np.sqrt((float(stats['x'])-xDest)**2 + (float(stats['y'])-yDest)**2) < VISIONRADIUS * 2):
            run('ElectricBoogalo','kirtyhurty','BRAKE')
            while(Moving):
                stats = parseStatus()
                if(float(stats['dx']) < 0.5 and float(stats['dy']) <0.5):
                    Moving = False
            

    return 1

#Shortest Path Math
def checkWormHoleCollision(xCur,yCur,xDest,yDest):
    print('thats this')
    for i in range(len(KNOWN_WORMHOLE_LOC)-1):
        wormX = float(KNOWN_WORMHOLE_LOC[i][0])
        wormY = float(KNOWN_WORMHOLE_LOC[i][1])
        rad = float(KNOWN_WORMHOLE_LOC[i][2])

        #shortest distance to line calculation
        distToLine = (np.abs((xDest-xCur)*wormX + (yCur-yDest)*wormY +
                            (xCur - xDest)*yCur + (yDest-yCur)*xCur)/
                      np.sqrt((xDest-xCur)**2 + (yCur-yDest)**2))
                      
        #check if inside circle
        if(distToLine < rad):
            angleChange = np.arcsin((2*rad)/np.sqrt((wormX-xCur)**2+(wormY-yCur)**2))
            return i, angleChange

    return -1, 0

#Moving Around Wormhole 
def avoidWormHole(check, angleChange, angle, xDest,yDest):
    run('ElectricBoogalo','kirtyhurty','BRAKE')
    time.sleep(1)
    while(check > -1):
        run('ElectricBoogalo','kirtyhurty','ACCELERATE ' + str(angle+angleChange) + ' 1')
        stats = parseStatus()
        check, differentAngle = checkWormHoleCollision(float(stats['x']),float(stats['y']),xDest,yDest)

    run('ElectricBoogalo','kirtyhurty','BRAKE')
    time.sleep(.2)

    return 0

In [38]:
run('ElectricBoogalo','kirtyhurty','STATUS')

STATUS_OUT 4956.287444277474 4984.5157721460655 -0.0018719105340572397 -4.684856116917456E-4  MINES 0 PLAYERS 11 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 5014.125832642741 5000.03485045098 9.860741689829796 1.8187116845466032E-6 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 5000.0 5000.0 0.0 0.0 BOMBS 0 WORMHOLES 0


In [37]:
status = parseStatus()
moveToPoint(float(status['x']),float(status['y']),5000,5000)
run('ElectricBoogalo','kirtyhurty','BRAKE')

BRAKE_OUT DONE
3.3980663275623613
thats this
3.3979145814787697
ACCELERATE_OUT DONE
3.3976937563303613
ACCELERATE_OUT DONE
3.397403618651647
ACCELERATE_OUT DONE
3.3972603290801695
ACCELERATE_OUT DONE
3.3970475693927145
ACCELERATE_OUT DONE
3.3968373364722524
ACCELERATE_OUT DONE
3.3966295500557653
ACCELERATE_OUT DONE
3.396424146486387
ACCELERATE_OUT DONE
3.396221063174891
ACCELERATE_OUT DONE
3.3960869269888594
ACCELERATE_OUT DONE
3.3958875573091927
ACCELERATE_OUT DONE
3.395625017712223
ACCELERATE_OUT DONE
3.3954304809400675
ACCELERATE_OUT DONE
3.395237904486982
ACCELERATE_OUT DONE
3.3950472229171025
ACCELERATE_OUT DONE
3.3948583703171193
ACCELERATE_OUT DONE
3.3946712712444738
ACCELERATE_OUT DONE
3.3944858613895192
ACCELERATE_OUT DONE
3.3942411666869505
ACCELERATE_OUT DONE
3.3940594562224855
ACCELERATE_OUT DONE
3.3938792338901287
ACCELERATE_OUT DONE
3.3936411312266754
ACCELERATE_OUT DONE
3.393405386540813
ACCELERATE_OUT DONE
3.3932300335288685
ACCELERATE_OUT DONE
3.3930558669439033
ACCELE

In [ ]:
range(len(KNOWN_WORMHOLE_LOC))

In [40]:
run('ElectricBoogalo','kirtyhurty','CONFIGURATIONS')

CONFIGURATIONS_OUT MAPWIDTH 10000 MAPHEIGHT 10000 CAPTURERADIUS 14.0 VISIONRADIUS 200.0 FRICTION 0.99 BRAKEFRICTION 0.987 BOMBPLACERADIUS 50.0 BOMBEFFECTRADIUS 15.0 BOMBDELAY 100 BOMBPOWER 15.0 SCANRADIUS 200.0 SCANDELAY 100


In [28]:
run('ElectricBoogalo','kirtyhurty','BRAKE')

BRAKE_OUT DONE


In [23]:
status = parseStatus()
yDest = 6000
xDest = 1000
np.arctan((yDest-float(status['y']))/(xDest-float(status['x'])))

-0.8143027859581448

In [21]:
status

{'bombs': [],
 'dx': -1.2650801528398812e-23,
 'dy': 1.5492763598402126e-39,
 'mines': [],
 'players': [],
 'wormholes': [],
 'x': 1943.799555253092,
 'y': 5000.0}